In [5]:
import pandas as pd
import numpy as np

#Define o nome das colunas 
r_colors = ['user_id', 'movie_id', 'rating']

# - `sep='\t'`: Usa tabulação como delimitador
# - `names=r_colors`: Define os nomes das colunas
# - `usecols=range(3)`: Usa apenas as três primeiras colunas do arquivo
ratings = pd.read_csv(r'C:\Users\jeffe\OneDrive\Documentos\Aquivos_de_C%C3%B3digo_1\u.data', sep='\t', names=r_colors, usecols=range(3))

#printa na tela as cinco primieras linhas do DF
print(ratings.head())


   user_id  movie_id  rating
0        0        50       5
1        0       172       5
2        0       133       1
3      196       242       3
4      186       302       3


In [6]:
#Agrupa o DF ratings pelo movie_ID e aplica funçoes de agregação na coluna rating
# Calcula o número de avaliações para cada 'movie_id'
# Calcula a média das notas (ratings) para cada 'movie_id'
movieProperties = ratings.groupby('movie_id').agg ({'rating': [np.size, np.mean]})
movieProperties.head()

C:\Users\jeffe\AppData\Local\Temp\ipykernel_6316\2611541783.py:4: FutureWarning: The provided callable <function mean at 0x000001E724AA3E20> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg ({'rating': [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [7]:
# Cria um novo DataFrame 'movieNumRatings' contendo a contagem de avaliações ('size') para cada filme ('movie_id')
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])

# Normaliza a contagem de avaliações para cada filme
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

# Exibe as primeiras cinco linhas do DataFrame normalizado 'movieNormalizedNumRatings'
print(movieNormalizedNumRatings.head())


              size
movie_id          
1         0.773585
2         0.222985
3         0.152659
4         0.356775
5         0.145798


In [8]:
movieDict = {}  # Inicializa um dicionário vazio para armazenar informações sobre filmes

with open(r'C:\Users\jeffe\OneDrive\Documentos\Aquivos_de_C%C3%B3digo_1\u.item') as f:
    temp = ''
    for line in f:  # Itera sobre cada linha do arquivo
        fields = line.rstrip('\n').split('|')  # Remove o '\n' e divide a linha em campos
        movie_ID = int(fields[0])  # Converte o primeiro campo para inteiro (ID do filme)
        name = fields[1]  # Extrai o nome do filme do segundo campo
        genres = fields[5:25]  # Extrai os gêneros do filme (20 campos)
        genres = list(map(int, genres))  # Converte os gêneros para inteiros
        
        # Atualiza o dicionário com uma tupla contendo o nome, gêneros, número normalizado de avaliações e propriedades das avaliações
        movieDict[movie_ID] = (
            name, 
            genres, 
            movieNormalizedNumRatings.loc[movie_ID].get('size'), 
            movieProperties.loc[movie_ID].rating
        )


In [9]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 np.float64(0.7735849056603774),
 size    452.000000
 mean      3.878319
 Name: 1, dtype: float64)

In [10]:
from scipy import spatial  # Importa o módulo 'spatial' da biblioteca 'scipy'

# Define uma função chamada 'computeDistance' que calcula a distância combinada entre dois filmes
def computeDistance(a, b):
    genresA = a[1]  # Extrai os gêneros do filme 'a'
    genresB = b[1]  # Extrai os gêneros do filme 'b'
    genreDistance = spatial.distance.cosine(genresA, genresB)  # Calcula a distância cosseno entre os gêneros
    popularityA = a[2]  # Extrai a popularidade do filme 'a'
    popularityB = b[2]  # Extrai a popularidade do filme 'b'
    popularityDistance = abs(popularityA - popularityB)  # Calcula a diferença absoluta de popularidade
    return genreDistance + popularityDistance  # Retorna a soma da distância de gêneros e popularidade

# Chama a função 'computeDistance' com os filmes de IDs 2 e 4
computeDistance(movieDict[2], movieDict[4])


np.float64(0.8004574042309892)

In [11]:
print (movieDict[2])
print (movieDict[4])


('GoldenEye (1995)', [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], np.float64(0.22298456260720412), size    131.000000
mean      3.206107
Name: 2, dtype: float64)
('Get Shorty (1995)', [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], np.float64(0.3567753001715266), size    209.000000
mean      3.550239
Name: 4, dtype: float64)


In [12]:
import operator  # Importa o módulo 'operator' para operações eficientes

def getNeighbors(movie_ID, K):
    distances = []  # Inicializa uma lista para armazenar as distâncias
    for movie in movieDict: 
        if movie != movie_ID:  # Garante que o filme atual não é o filme de interesse
            dist = computeDistance(movieDict[movie_ID], movieDict[movie])  # Calcula a distância entre os filmes
            distances.append((movie, dist))  # Adiciona a tupla (ID do filme, distância) à lista de distâncias
    distances.sort(key=operator.itemgetter(1))  # Ordena as distâncias em ordem crescente
    neighbors = []  # Inicializa a lista de vizinhos
    for x in range(K):  # Seleciona os K vizinhos mais próximos
        neighbors.append(distances[x][0])  # Adiciona o ID do vizinho à lista de vizinhos
    return neighbors  # Retorna a lista de vizinhos

K = 10  # Define o número de vizinhos a serem encontrados
avgRating = 0  # Inicializa a soma das avaliações dos vizinhos
neighbors = getNeighbors(1, K)  # Encontra os 3 vizinhos mais próximos do filme com ID 1
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]  # Adiciona a avaliação do vizinho à soma
    print(movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))  # Imprime o nome e a avaliação do vizinho

avgRating /= float(K)  # Calcula a média das avaliações dos vizinhos


Liar Liar (1997) size    485.000000
mean      3.156701
Name: 294, dtype: float64
Aladdin (1992) size    219.000000
mean      3.812785
Name: 95, dtype: float64
Willy Wonka and the Chocolate Factory (1971) size    326.000000
mean      3.631902
Name: 151, dtype: float64
Monty Python and the Holy Grail (1974) size    316.000000
mean      4.066456
Name: 168, dtype: float64
Full Monty, The (1997) size    315.000000
mean      3.926984
Name: 269, dtype: float64
George of the Jungle (1997) size    162.000000
mean      2.685185
Name: 259, dtype: float64
Beavis and Butt-head Do America (1996) size    156.000000
mean      2.788462
Name: 240, dtype: float64
Birdcage, The (1996) size    293.000000
mean      3.443686
Name: 25, dtype: float64
Home Alone (1990) size    137.000000
mean      3.087591
Name: 94, dtype: float64
Aladdin and the King of Thieves (1996) size    26.000000
mean     2.846154
Name: 422, dtype: float64


In [13]:
avgRating

size    243.500000
mean      3.344591
Name: 294, dtype: float64

In [14]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 np.float64(0.7735849056603774),
 size    452.000000
 mean      3.878319
 Name: 1, dtype: float64)

Activity

Que efeitos os diferentes valores de K tem nos resultados. Nossa metrica de distancia tambem foi um pouco arbitraria, apenas pegamos a distancias do cosseno entre os generos e adicionamos a diferença entre as pontuaçoes de popularidade normalizadas. Consegue melhorar?

In [16]:
import operator

# Calcula a distância entre o filme alvo e os demais
def ComputeDistance(movieA, movieB):
    # Ajusta a metrica para priorizar a diferenca de popularidade
    popularity_diff = abs(movieA[2] - movieB[2])  # Diferenca de popularidade
    genre_similarity = sum([a * b for a, b in zip(movieA[1], movieB[1])])  # Generos similares
    return popularity_diff - genre_similarity  # Maior diferença de popularidade menos a distância

def getNeighbors(movieID, K, distance_metric):
    distances = []
    for movie in movieDict:
        if movie != movieID:
            dist = distance_metric(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    # Ordena os filmes pela distância, em ordem crescente (menor distância é mais próximo)
    distances.sort(key=operator.itemgetter(1))
    neighbors = [distances[x][0] for x in range(K)]
    return neighbors

K = 10
# Chama a função getNeighbors para encontrar os K vizinhos mais próximos
neighbors = getNeighbors(1, K, ComputeDistance)
# Calcula a média das avaliações
avgRating = sum([movieDict[neighbor][3] for neighbor in neighbors]) / K
#Imprime o nome  e a avaliação dos vizinhos
for neighbor in neighbors:
    print(movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
print(f"Average Rating with improved metric: {avgRating:.2f}")

Aladdin (1992) size    219.000000
mean      3.812785
Name: 95, dtype: float64
Space Jam (1996) size    93.000000
mean     2.774194
Name: 820, dtype: float64
Hercules (1997) size    66.000000
mean     3.515152
Name: 993, dtype: float64
Aladdin and the King of Thieves (1996) size    26.000000
mean     2.846154
Name: 422, dtype: float64
Goofy Movie, A (1995) size    20.0
mean     2.9
Name: 1219, dtype: float64
Willy Wonka and the Chocolate Factory (1971) size    326.000000
mean      3.631902
Name: 151, dtype: float64
Lion King, The (1994) size    220.000000
mean      3.781818
Name: 71, dtype: float64
Babe (1995) size    219.000000
mean      3.995434
Name: 8, dtype: float64
Beauty and the Beast (1991) size    202.000000
mean      3.792079
Name: 588, dtype: float64
Mary Poppins (1964) size    178.000000
mean      3.724719
Name: 419, dtype: float64


TypeError: unsupported format string passed to Series.__format__

In [17]:
avgRating

size    156.900000
mean      3.477424
dtype: float64

In [18]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 np.float64(0.7735849056603774),
 size    452.000000
 mean      3.878319
 Name: 1, dtype: float64)